In [11]:
import numpy as np

In [12]:
class FireFly:
    def __init__(self, N: int) -> None:
        """
        N: число городов (число вершин в графе)
        """
        self.N = N
        self.genotype = np.array([
            np.random.randint(0, N - i) for i in range(N)
        ])

    def __repr__(self):
        return f'{self.genotype}'
    
    def get_route(self) -> np.ndarray:
        L = list(range(self.N))
        return np.array([L.pop(gene) for gene in self.genotype])

In [13]:
f = FireFly(5)
f

[0 0 1 0 0]

In [14]:
f.get_route()

array([0, 1, 3, 2, 4])

In [15]:
class Environment:
    ...